In [2]:
!pip install sacremoses --q

In [3]:
import os
import re
import sys
import unicodedata
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sacremoses import MosesPunctNormalizer
from transformers import T5ForConditionalGeneration, T5ForConditionalGeneration, T5Tokenizer

In [22]:
DATA_PATH = '/kaggle/input/'
DATA_FOLDER_PATH = os.path.join(DATA_PATH, 'aromanian')
T5_RUP_RO_PATH =  os.path.join(DATA_PATH, 't5-rup-ro')
T5_RUP_STD_RO_PATH = os.path.join(DATA_PATH, 't5-rup-ro')
T5_RUP_CON_RO_PATH = os.path.join(DATA_PATH, 't5-rup-con-ro')
T5_EN_RO_PATH = os.path.join(DATA_PATH, 't5-en-ro')
T5_ES_RO_PATH =  os.path.join(DATA_PATH, 't5-es-ro')
T5_FR_RO_PATH =  os.path.join(DATA_PATH, 't5-fr-ro')

FILE_NAME = 'Tales_processed.csv'
DATA_LANGUAGES = {
    'rup': 'Aromanian General Dialect',
    'rup_std': 'Aromanian Standard Dialect',
    'rup_cun': 'Aromanian Regional Dialect',
    'en': 'English',
    'es': 'Spanish',
    'fr': 'French'
}

In [5]:
path = os.path.join(DATA_FOLDER_PATH, FILE_NAME)
translations_df = pd.read_csv(path, index_col=[0]).drop(['ro', 'split'], axis=1)
translations_df

,rup,rup_std,rup_cun,en,es,fr
0,teoria și practica,teoria și practica,teoria shi practica,theory and practice,teoría y práctica,théorie et pratique
1,"sănt îndoauă învețuri, cari si amintă mași cu ...","sănt îndoauă învețuri, cari si amintă mași cu ...","sãnt ãndoauã ãnvetsuri, cari si amintã mashi c...",there are some skills you gain just by reading...,hay algunas habilidades que se adquieren con s...,il y a des compétences que l'on acquiert simpl...
2,"ași zbura unoară un auș cu minte, cătră-ndoi t...","ași zbura unoară un auș cu minte, cătră-ndoi t...","ashi zbura unoarã un aush cu minte, cãtrã-ndoi...",so said a wise old man once to young people wh...,así se lo dijo una vez un viejo sabio a unos j...,c'est ce qu'a dit un jour un vieux sage à des ...
3,"ca s-lă da s-achicăsească ți va dzîșu pîn aua,...","ca s-lă da s-achicăsească ți va d̦îșu pîn aua,...",ca s-lã da s-achicãseascã tsi va dzãshu pãn au...,"to understand what i have told you so far, con...",para comprender lo que te he dicho hasta ahora...,pour comprendre ce que je vous ai dit jusqu'à ...
4,un amiră cu multe dhrăñi în cap și alăgat căt ...,un amiră cu multe dhrăńi în cap și alăgat căt ...,un amirã cu multe dhrãnji ãn cap shi alãgat cã...,"some wise emperor and a wise man, not a joke s...","algún sabio emperador -y un sabio, no un chist...","un sage empereur et un sage, ce n'est pas une ..."
...,...,...,...,...,...,...
2106,vimtul lișor lă hărisi fața... și-l'i plîmse.....,vimtul lișor lă hărisi fața... și-ľi plîmse......,vimtul lishor lã hãrisi fatsa... shi-lji plãms...,the wind gently caressed their cheeks... it's ...,el viento acariciaba suavemente sus mejillas.....,le vent caresse doucement leurs joues... c'est...
2107,hoara s-avea umplută di luñină. dina cu vira ș...,hoara s-avea umplută di luńină. dina cu vira ș...,hoara s-avea umplutã di lunjinã. dina cu vira ...,light pours over the village. dinu and vira ar...,la luz se derrama sobre la aldea. dinu y vira ...,la lumière se répand sur le village. dinu et v...
2108,"la boațea aistă, hoara arăsuna jilos... și-apo...","la boațea aistă, hoara arăsuna jilos... și-apo...","la boatsea aistã, hoara arãsuna jilos... shi-a...","at the ringing of the bell, the village rang d...","al toque de campana, el pueblo sonó desierto.....","au son de la cloche, le village retentit déser..."
2109,"vira s-turnă acasă... cîntă, arăse... s-alăxi ...","vira s-turnă acasă... cîntă, arăse... s-alăxi ...","vira s-turnã acasã... cãntã, arãse... s-alãxi ...","vira returned home... singing, laughing... she...","vira volvió a casa... cantando, riendo... se p...","vira est rentrée chez elle... elle chante, ell..."


In [10]:
data = []
for language in translations_df.columns:
    for text in translations_df[language]:
        data.append({'text': text, 'language': language})
language_data = pd.DataFrame(data)
language_data

,text,language
0,teoria și practica,rup
1,"sănt îndoauă învețuri, cari si amintă mași cu ...",rup
2,"ași zbura unoară un auș cu minte, cătră-ndoi t...",rup
3,"ca s-lă da s-achicăsească ți va dzîșu pîn aua,...",rup
4,un amiră cu multe dhrăñi în cap și alăgat căt ...,rup
...,...,...
12661,le vent caresse doucement leurs joues... c'est...,fr
12662,la lumière se répand sur le village. dinu et v...,fr
12663,"au son de la cloche, le village retentit déser...",fr
12664,"vira est rentrée chez elle... elle chante, ell...",fr


In [11]:
X_train, X_test, y_train, y_test = train_test_split(language_data['text'], language_data['language'], test_size=0.2, random_state=42)

In [12]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [13]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=5000)

In [14]:
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8681925808997633
              precision    recall  f1-score   support

          en       1.00      0.97      0.99       418
          es       0.96      0.98      0.97       436
          fr       0.99      0.97      0.98       375
         rup       0.64      0.73      0.68       421
     rup_cun       0.94      0.93      0.94       431
     rup_std       0.73      0.64      0.68       453

    accuracy                           0.87      2534
   macro avg       0.87      0.87      0.87      2534
weighted avg       0.87      0.87      0.87      2534



In [25]:
class TranslationModel:
    def __init__(self, model_path, source_lang_code, source_language, target_lang_code='ro', target_language='Romanian'):
        self.source_lang_code = source_lang_code
        self.source_language = source_language
        
        self.target_lang_code = target_lang_code
        self.target_language = target_language
        
        self.model_path = model_path 
        self.model = T5ForConditionalGeneration.from_pretrained(self.model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(self.model_path)

        self.language_map = {
            'ro': 'ro',
            'rup': 'ro',  
            'rup_std': 'ro',  
            'rup_cun': 'ro',
            'en': 'en',
            'es': 'es',
            'fr': 'fr'
        }
        
        self.replace_nonprint = self.__get_non_printing_char_replacer("")
        
    def __get_non_printing_char_replacer(self, replace_by: str = " "):
        non_printable_map = {
            ord(c): replace_by
            for c in (chr(i) for i in range(sys.maxunicode + 1))
            if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
        }

        def replace_non_printing_char(text) -> str:
            return text.translate(non_printable_map)

        return replace_non_printing_char
    
    def __process_text(self, text):
        mpn = MosesPunctNormalizer(lang=self.language_map[self.source_lang_code])
        mpn.substitutions = [(re.compile(r), sub) for r, sub in mpn.substitutions]

        clean = mpn.normalize(text)
        clean = self.replace_nonprint(clean)
        # ex: replace '𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞' with 'Francesca'
        clean = unicodedata.normalize("NFKC", clean)
        clean = clean.replace('"', '').replace(' - ', ' ')
        clean = clean.strip()

        return clean
        
        
    def translate(self, text):
        text = self.__process_text(text)
        
        prefix = f"translate {self.source_language} to {self.target_language}: "
        
        input_ids = self.tokenizer(prefix + text, return_tensors="pt").input_ids
        output_ids = self.model.generate(input_ids=input_ids)
        output = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        
        return output

In [26]:
translation_models = {
    'rup': TranslationModel(T5_RUP_RO_PATH, 'rup', 'Aromanian General Dialect'), 
    'rup_std': TranslationModel(T5_RUP_RO_PATH, 'rup_std', 'Aromanian Standard Dialect'),
    'rup_cun': TranslationModel(T5_RUP_CON_RO_PATH, 'rup_cun', 'Aromanian Regional Dialect'),
    'en': TranslationModel(T5_EN_RO_PATH, 'en', 'English'),
    'es': TranslationModel(T5_ES_RO_PATH, 'es', 'Spanish'),
    'fr': TranslationModel(T5_ES_RO_PATH, 'es', 'French'),
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
while True:
    text_input = input("Input: ")
    text_tfidf = tfidf_vectorizer.transform([text_input])
    lang_predict = model.predict(text_tfidf)[0]
    print(f"-> Language predicted: {DATA_LANGUAGES[lang_predict]} [{lang_predict}]")
    translation = translation_models[lang_predict].translate(text_input)
    print(f"-> Translation: {translation} \n")

Input:  Shi lu-avdzã Dumnidzãlu!


-> Language predicted: Aromanian Regional Dialect [rup_cun]
-> Translation: și îl așteaptă pe dumnezeu! 



Input:  Ațea de-aeri, țe-ńi d̦îțeai!


-> Language predicted: Aromanian Standard Dialect [rup_std]
-> Translation: a fost de-aia, ce-mi zise! 



Input:  Cățe, tată, tut uhted̦, plînd̦î și stai învirinat?


...

-> Language predicted: English [en]
-> Translation: l-au omorât și ei și l-au trimis 



Input:  ¿CÓMO ES QUE NO SON GITANOS?


-> Language predicted: Spanish [es]
-> Translation: cum e de ce n-am știre de ea? 



Input:  Mettez-moi des ailes, comme un cerf-volant 


-> Language predicted: French [fr]
-> Translation: te-mi duci de aile, ca un cerf 

